In [118]:
import os
from pathlib import Path
from aocd.models import Puzzle
import collections
import numpy as np
import re
from statistics import median, mean
import math
import queue
import itertools
import more_itertools
infinite_defaultdict = lambda: defaultdict(infinite_defaultdict)
from copy import copy
import functools
import heapq
import operator
import tqdm

# Day 1

In [ ]:
puzzle = Puzzle(2022, 1)

In [ ]:
elf_snaks = [sum(map(int, line.split("\n"))) for line in puzzle.input_data.split("\n\n")]

## Part 1 

In [ ]:
res_a = max(elf_snaks)
res_a

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
res_b = sum(sorted(elf_snaks)[-3:])
res_b

In [ ]:
puzzle.answer_b = res_b

# Day 2

In [ ]:
puzzle = Puzzle(2022, 2)

In [ ]:
data = [line.split(" ") for line in puzzle.input_data.split("\n")]

## Part 1 

In [ ]:
score_mapping = {k: idx+1 for idx, k in enumerate("ABC")}
letter_mapping = dict(zip("XYZ","ABC"))
def parse_pair(e1, e2, mapped=False):
    if not mapped:
        e2 = letter_mapping[e2]
    s1, s2 = score_mapping[e1], score_mapping[e2]
    
    res = (ord(e1) - ord(e2) + 1) % 3
    
    return res * 3 + s1, s2 + (2 - res) * 3

In [ ]:
res_a = sum(parse_pair(*pair)[1] for pair in data)
res_a

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
outcome_mapping = {k: idx-1 for idx, k in enumerate("XYZ")}
def parse_outcome(e1, out_come):
    e2 = chr((ord(e1) - ord("A") + outcome_mapping[out_come]) % 3 + ord("A"))
    return parse_pair(e1, e2, mapped=True)

In [ ]:
res_b = sum(parse_outcome(*pair)[1] for pair in data)
res_b

In [ ]:
puzzle.answer_b = res_b

# Day 3

In [ ]:
puzzle = Puzzle(2022, 3)

In [ ]:
data = puzzle.input_data.split("\n")
data

In [ ]:
mapping = {chr(ord('a') + i): i + 1 for i in range(26)}
mapping.update({chr(ord('A') + i): i + 26 + 1 for i in range(26)})

## Part 1 

In [ ]:
res_a = sum(mapping[elem] for line in data for elem in set(line[:len(line)//2]) & set(line[len(line)//2:]))
res_a

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
answer_b = sum(mapping[elem] for l1, l2, l3 in zip(*[iter(data)]*3) for elem in set(l1) & set(l2) & set(l3))
answer_b    

In [ ]:
puzzle.answer_b = answer_b

# Day 4

In [9]:
puzzle = Puzzle(2022, 4)

In [40]:
data = puzzle.input_data.split("\n")
pairs = [line.split(",") for line in data]

In [ ]:
pair_pattern = re.compile(r"(?P<n1>\d+)-(?P<n2>\d+)")
def pair_to_set(pair):
    match = pair_pattern.match(pair)
    start, end = map(int, match.groups())
    return set(range(start, end + 1))    

## Part 1 

In [42]:
def pair_included(p1, p2):
    s1, s2 = pair_to_set(p1), pair_to_set(p2)
    return s1 <= s2 or s1 >= s2

In [43]:
answer_a = sum(pair_included(*p) for p in pairs)
answer_a

518

In [39]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [46]:
def pair_overlap(p1, p2):
    s1, s2 = pair_to_set(p1), pair_to_set(p2)
    return bool(s1 & s2)

In [47]:
answer_b = sum(pair_overlap(*p) for p in pairs)
answer_b

909

In [49]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 4! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 5

In [4]:
puzzle = Puzzle(2022, 5)

In [33]:
data = puzzle.input_data

In [102]:
instruction_pattern = re.compile(r"move (?P<number>\d+) from (?P<start>\d+) to (?P<end>\d+)")
class SupplyStacks:
    def __init__(self, data):
        queues, instructions = data.split("\n\n")
        # transpose
        queues = [''.join(s).strip() for s in zip(*queues.split("\n")) if re.match(r"\w+", ''.join(s).strip())]
        
        self.queues = {}
        # Create LIFO
        for q in queues:
            new_q = queue.LifoQueue()
            for s in q[-2::-1]:
                new_q.put(s)
            self.queues[q[-1]] = new_q
        
        # Parse instructions
        self.instructions = [instruction_pattern.match(instruction).groupdict() for instruction in instructions.split("\n")]
        
    def part_1(self):
        for instruction in self.instructions:
            for _ in range(int(instruction["number"])):
                self.queues[instruction["end"]].put(self.queues[instruction["start"]].get())
    
    def answer_a(self):
        self.part_1()
        return self._join_queues()
    
    def part_2(self):
        temporary_queue = queue.LifoQueue()
        for instruction in self.instructions:
            for _ in range(int(instruction["number"])):
                temporary_queue.put(self.queues[instruction["start"]].get())
                
            for _ in range(int(instruction["number"])):
                self.queues[instruction["end"]].put(temporary_queue.get())
                
    def answer_b(self):
        self.part_2()
        return self._join_queues()
    
    def _join_queues(self):
        return "".join(q.get() for q in self.queues.values())

## Part 1 

In [96]:
ss = SupplyStacks(data)

In [94]:
answer_a = ss.answer_a()
answer_a

'FJSRQCFTN'

In [89]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [103]:
ss = SupplyStacks(data)

In [104]:
answer_b = ss.answer_b()
answer_b

'CJVLJQPHS'

In [105]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 5! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 6

In [12]:
puzzle = Puzzle(2022, 6)

In [13]:
data = puzzle.input_data

## Part 1 

In [15]:
def get_position_message(data, length):
    for idx, chars in enumerate(more_itertools.sliding_window(data, length)):
        if len(set(chars)) == length:
            return idx + length
    
    raise ValueError("should not happen")

In [16]:
answer_a = get_position_message(data, 4)
answer_a

1912

In [16]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [17]:
answer_b = get_position_message(data, 14)
answer_b

2122

In [18]:
puzzle.answer_b = answer_b

# Day 7

In [3]:
puzzle = Puzzle(2022, 7)

In [4]:
data = puzzle.input_data.split("\n")

In [6]:
directrories_score = collections.defaultdict(int)
actual_dir = collections.deque("/")

for line in data:
    match line.split():
        # commands
        case '$', 'ls':
            pass
        case '$', 'cd', '/':
            actual_dir = collections.deque("/")
        case '$', 'cd', '..':
            actual_dir.pop()
        case '$', 'cd', dir_name:
            actual_dir.append(f"{dir_name}/")
        # after ls
        case 'dir', _:
            pass
        case size, _:
            # Accumulate for concatening all elements of actual_dir
            for directory in itertools.accumulate(actual_dir):
                directrories_score[directory] += int(size)

## Part 1 

In [8]:
answer_a = sum(val for val in directrories_score.values() if val <= 100000)
answer_a

1297683

In [109]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [9]:
answer_b = min(val for val in directrories_score.values() if 30000000 <= 70000000 - directrories_score['/'] + val)
answer_b

5756764

In [10]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 7! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 8

In [195]:
puzzle = Puzzle(2022, 8)

In [122]:
data = np.array([[int(number) for number in line] for line in puzzle.input_data.split("\n")])

In [123]:
len_x, len_y = data.shape

## Part 1 

In [136]:
total_tree = np.ones((len_x,len_y), dtype=int)
total_tree[1:-1,1:-1] = 0

for x in range(1, len_x - 1):
    for y in range(1, len_y - 1):
        number = data[x, y]
        if (
            np.all(data[:x, y] < number)
            or np.all(data[x+1:, y] < number)
            or np.all(data[x, :y] < number)
            or np.all(data[x, y+1:] < number)
         ):
            total_tree[x, y] = 1

In [137]:
answer_a = np.sum(total_tree)
answer_a

1829

In [121]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [193]:
total_tree = np.ones((len_x,len_y), dtype=int)
total_tree[1:-1,1:-1] = 0

for x in range(1, len_x-1):
    for y in range(1, len_y-1):
        number = data[x, y]
        
        a = 0
        for new_n in data[x-1::-1, y]:
            a += 1
            if new_n >= number:
                break
                
        b = 0
        for new_n in data[x+1:, y]:
            b += 1
            if new_n >= number:
                break
        
        c = 0
        for new_n in data[x, y-1::-1]:
            c += 1
            if new_n >= number:
                break
                
        d = 0
        for new_n in data[x, y+1:]:
            d += 1
            if new_n >= number:
                break
                
        total_tree[x, y] = a * b * c * d       

In [194]:
answer_b = np.max(total_tree)
answer_b

291840

In [192]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 8! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 9

In [2]:
puzzle = Puzzle(2022, 9)

In [288]:
data = puzzle.input_data

In [289]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @property
    def as_tuple(self):
        return self.x, self.y
    
    @classmethod
    def from_tuple(cls, coord):
        return Point(*coord)
                   
    @classmethod
    def create_vector(cls, direction):
        match direction:
            case "U":
                return cls(1, 0)
            case "R":
                return cls(0, 1)
            case "D":
                return cls(-1, 0)
            case "L":
                return cls(0, -1)
            case "_":
                raise ValueError("unkown direction")
        
    def one_tail_near(self, other_point):
        return abs(self.x - other_point.x) <= 1 and abs(self.y - other_point.y) <= 1
    
    def sign(self):
        return Point(np.sign(self.x), np.sign(self.y))
    
    def __sub__(self, other_point):
        return Point(self.x - other_point.x, self.y - other_point.y)
    
    def __isub__(self, other_point):
        return self - other_point
    
    def __neg__(self):
        return Point(-self.x, -self.y)
    
    def __add__(self, other_point):
        return Point(self.x + other_point.x, self.y + other_point.y)
    
    def __iadd__(self, other_point):
        return self + other_point
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __repr__(self):
        return f"Point{self}"

In [290]:
class RopeGame:
    def __init__(self, data, tail_lenght=1, start=(0,0)):
        self.instructions = [line.split(" ") for line in data.split("\n")]
        self.knots = [Point.from_tuple(start) for _ in range(tail_lenght + 1)]
        self.knots_positions = [set([start]) for _ in range(tail_lenght + 1)]
    
    def parse_instruction(self, instruction):
        direction, value = instruction
        vector = Point.create_vector(direction)
        
        for _ in range(int(value)):
            self.update_knot(0, self.knots[0] + vector)
            
            for idx, knot in list(enumerate(self.knots))[1:]:
                prev_knot = self.knots[idx-1]
                if knot.one_tail_near(prev_knot):
                    break
                
                deplacement = (prev_knot - knot).sign()
                self.update_knot(idx, knot + deplacement)
                
    def run(self):
        for instruction in self.instructions:
            self.parse_instruction(instruction)
            
    def update_knot(self, pos, new_knot):
        self.knots[pos] = new_knot
        self.knots_positions[pos].add(new_knot.as_tuple)
    
    @property
    def answer_a(self):
        return len(self.knots_positions[1])
    
    @property
    def answer_b(self):
        return len(self.knots_positions[-1])
    

## Part 1 

In [291]:
rg = RopeGame(data)
rg.run()
rg.answer_a

6057

In [292]:
puzzle.answer_a = rg.answer_a

## Part 2

In [293]:
rg = RopeGame(data, 9)
rg.run()

In [294]:
rg.answer_a

6057

In [295]:
rg.answer_b

2514

In [296]:
puzzle.answer_b = rg.answer_b

# Day 10

In [155]:
puzzle = Puzzle(2022, 10)

In [156]:
data = puzzle.input_data.split("\n")

In [157]:
def test_cycle(x, cycle):
    if cycle in {20, 60, 100, 140, 180, 220}:
        print(cycle, x)
        return cycle * x
    return 0

## Part 1 

In [158]:
x = 1
cycle = 0
answer_a = 0

for instruction in data:
    match instruction.split(" "):
        case ["noop"]:
            cycle +=1
            answer_a += test_cycle(x, cycle)
        case "addx", value:
            cycle += 1
            answer_a += test_cycle(x, cycle)
            cycle += 1
            answer_a += test_cycle(x, cycle)
            x += int(value)
        case _:
            print(instruction)
            raise ValueError("")
answer_a

20 29
60 17
100 21
140 21
180 21
220 17


14160

In [107]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [159]:
def draw(x, cycle):
    if x <= (cycle % 40) <= x+2:
        return "#"
    return "."

In [160]:
def print_map(answer_b):
    for chunk in more_itertools.batched(answer_b, 40):
        print("".join(chunk))

In [161]:
x = 1
cycle = 0
answer_b = []

for instruction in data:
    match instruction.split(" "):
        case ["noop"]:
            cycle +=1
            answer_b.append(draw(x, cycle))
        case "addx", value:
            cycle += 1
            answer_b.append(draw(x, cycle))
            cycle += 1
            answer_b.append(draw(x, cycle))
            x += int(value)
        case _:
            print(instruction)
            raise ValueError("")


In [162]:
print_map(answer_b)

###....##.####.###..###..####.####..##.#
#..#....#.#....#..#.#..#.#....#....#..#.
#..#....#.###..#..#.#..#.###..###..#....
###.....#.#....###..###..#....#....#...#
#.#..#..#.#....#.#..#....#....#....#..#.
#..#..##..####.#..#.#....####.#.....##..


In [154]:
puzzle.answer_b = "RJERPEFC"

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 10! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 11

In [76]:
puzzle = Puzzle(2022, 11)

In [77]:
data = puzzle.input_data

In [67]:
data = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [101]:
class Monkey:
    def __init__(self, name,items, operation, test_val, monkey_true, monkey_false):
        self.item_inspected = 0
        self.name = name,
        self.items = items
        self.operation = operation
        self.test_val = test_val
        self.monkey_true = monkey_true
        self.monkey_false = monkey_false
    
    def inspect(self, old, p1=True):
        self.item_inspected += 1
        item = eval(self.operation)
        if p1:
            item = item // 3
        return item
        
    def test(self, item):
        monkey_dest = self.monkey_true if item % self.test_val == 0 else self.monkey_false
        return monkey_dest, item
    
    def add_item(self, item):
        self.items.append(item)
    
    def do_round(self, p1=True):
        for item in self.items:
            item = self.inspect(item, p1=p1)
            yield self.test(item)
        self.items = []

In [131]:
class MonkeyManager:
    def __init__(self, data):
        # Create monkey
        self.monkeys = []
        self.test_total = 1
        self.data = data
        
        self.create()
    
    def create(self):

        for monkey_data in self.data.split("\n\n"):
            name,  items, operation, test, m_true, m_false = monkey_data.split("\n")
            name,  items, operation, test, m_true, m_false = monkey_data.split("\n")
            name = name.strip()
            items = list(map(int, number_pattern.findall(items)))
            operation = operation.split("= ")[1]
            test = int(number_pattern.search(test).group())
            self.test_total *= test
            m_true = int(number_pattern.search(m_true).group())
            m_false = int(number_pattern.search(m_false).group())
            new_monkey = Monkey(name, items, operation, test, m_true, m_false)
            self.monkeys.append(new_monkey)
            
    def do_round(self, p1=True):
        
        for monkey in self.monkeys:
            for monkey_dest, item in monkey.do_round(p1):
                self.monkeys[monkey_dest].add_item(item % self.test_total)
                
    def answer_a(self):
        self.reset()
        for i in range(20):
            self.do_round(p1=True)
        
        return self.monkey_inspections(2)
    
    
    def answer_b(self):
        self.reset()
        for i in tqdm.tqdm(range(10000)):
            self.do_round(p1=False)
        
        return self.monkey_inspections(2)
    
    
    def monkey_inspections(self, n):
        m_i = [monkey.item_inspected for monkey in self.monkeys]
        m_i = heapq.nlargest(2, m_i)
        return functools.reduce(operator.mul, m_i, 1)
        
    def reset(self):
        self.monkeys = []
        self.test_total = 1       
        self.create()

In [132]:
mm = MonkeyManager(data)

## Part 1 

In [133]:
answer_a = mm.answer_a()
answer_a

66802

In [113]:
puzzle.answer_a = answer_a

## Part 2

In [134]:
answer_b = mm.answer_b()
answer_b

100%|███████████████████████████████████| 10000/10000 [00:04<00:00, 2329.57it/s]


21800916620

In [135]:
puzzle.answer_b = answer_b